# DSCI100 Project Final Report

In [2]:
library(tidyverse)
library(repr)
library(tidymodels)
library(dplyr)

url = "http://drive.google.com/uc?rxport-download&id=19dtTv9I4hUdTKPBrM1QgI3A0ru68ssds"
players <- read_csv(url)
dim(players); head(players)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.6     ✔ rsample      1.2.1
✔ dials        1.3.0     ✔ tune         1.1.2
✔ infer        1.0.7     ✔ workflows    1.1.4
✔ modeldata    1.4.0     ✔ workflowsets 1.0.1
✔ parsnip      1.2.1     ✔ yardstick    1.3.1
✔ recipes      1.1.0     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

[1] 196   7

experience,subscribe,hashedEmail,played_hours,name,gender,Age
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9
Veteran,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17
Veteran,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17
Amateur,TRUE,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4fa7a5a659ff443a0eb5,0.7,Flora,Female,21
Regular,TRUE,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb0af4d48fcce2420f3e,0.1,Kylie,Male,21
Amateur,TRUE,f58aad5996a435f16b0284a3b267f973f9af99e7a89bee0430055a44fa92f977,0.0,Adrian,Female,17


## Introduction

Description background info

**Research Question**: 

The **`players`** dataset contains **196 observations and 7 variables.**  
    Variables include: 

+ **`experience`**, character, experience level of the player, categories include `Beginner`, `Regular`, `Amateur`, `Veteran`, `Pro`. 
+ **`subscribe`**, logical, whether player is subcribed to game-related newsletter, either `True` or `False`.
+ **`hashedEmail`**, character, records the hashed Email address of players.
+ **`played_hours`**, double, records the hours of video game played, records minimum of 0 (hours) and maximum 223.1 (hours). 
+ **`name`**, character, first name of the player.
+ **`gender`**, character, gender of the player, include `Male`, `Female`, `Non-Binary`, `Prefer not to say`, `Agender`, `Two-Spirited`, and `Other`. 
+ **`Age`**, double, Age of the player, records minimum of 9 (years old) and maximum of 58 (years old).


## Methods and Results

*describe the methods you used to perform your analysis from beginning to end that narrates the analysis code.*

For my Question, I plan to use **Knn-Classification** on my predicators, `Gender`, `Age`, `Experience`, `hours_played` to predict when `subscribe` = `True`. I chose Knn-Classification because it predicts a categorical response variable, just like our `subscribed`, which is split into two categories of `true` and `false`. 

This plan presents certain limitations like class imbalance in `gender` where the `Male` category has a large majority and is therefore more likely to be predicted out of probability. Furthermore, other limitations of the Knn-classification algorithm includes biases towards variables with a larger numerical range; to resolve this issue, I will standardize before I split the data into testing and training sets. I would need to convert `experience` and `gender` into a numeric vector before I start. 

I will **split the data into 75% training and 25% testing** sets. 
Prior to the classification, I will:  

1. standardize the data
2. split into training/ testing
3. cross-validate and k-tuning to find best K, with the highest accuracy.
4. use on testing set to find accuracy and precision of model
5. evaluate excellence of model, consider if further revisions of model is needed.



your report should include code which:  
loads data   
wrangles and cleans the data to the format necessary for the planned analysis  
performs a summary of the data set that is relevant for exploratory data analysis related to the planned analysis   
creates a visualization of the dataset that is relevant for exploratory data analysis related to the planned analysis  
performs the data analysis  
creates a visualization of the analysis   
note: all figures should have a figure number and a legend  

In [24]:
players <- players|>
        as_factor

ERROR: Error in parse(text = x, srcfile = src): <text>:2:0: unexpected end of input
1: players <- players|>
   ^


In [25]:
players_recipe <- recipe(subscribe ~ played_hours + Age , data = players_split) |>
   step_center(all_predictors())|>
   step_scale(all_predictors()) 

In [26]:
players_split <- initial_split(players, prop = 0.75, strata = subscribe)  
players_training <- training(players_split)   
players_testing <- testing(players_split)

In [27]:
knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
      set_engine("kknn") |>
      set_mode("classification")

In [28]:
players_fit <- workflow() |>
      add_recipe(players_recipe)|>
      add_model(knn_spec) |>
      fit(data = players_training)

ERROR: [1m[33mError[39m in `check_outcome()`:[22m
[33m![39m For a classification model, the outcome should be a `factor`, not a `logical`.


In [18]:
players_test_predictions <- predict(players_fit, players_testing) |>
      bind_cols(players_testing)

ERROR: Error in eval(expr, envir, enclos): object 'players_fit' not found


In [ ]:
players_prediction_accuracy <- players_test_predictions |>
        metrics(truth = .pred_class, estimate = fruit_name)   

## Discussion

summarize what you found  
discuss whether this is what you expected to find  
discuss what impact could such findings have  
discuss what future questions could this lead to

## References (optional)

may include references if necessary, as long as they all have a consistent citation style.